In [4]:
import gradio as gr
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

# Constants
STARTING_CAPITAL = 100000
COMMISSION = 20
MAX_POSITION_SIZE = 0.2
MIN_TRADE_AMOUNT = 1000

# Global state
portfolio = {}
transactions = []
capital = STARTING_CAPITAL

# NIFTY 50 Stocks (short list)
stocks = {
    "Reliance": "RELIANCE.NS",
    "TCS": "TCS.NS",
    "Infosys": "INFY.NS",
    "HDFC Bank": "HDFCBANK.NS",
    "ICICI Bank": "ICICIBANK.NS",
    "HUL": "HINDUNILVR.NS",
    "SBI": "SBIN.NS",
    "Kotak Bank": "KOTAKBANK.NS",
    "Airtel": "BHARTIARTL.NS",
    "Wipro": "WIPRO.NS"
}

def get_signals(ticker):
    try:
        df = yf.download(ticker, period="60d", progress=False)
        if df.empty or len(df) < 15:
            return None, "Insufficient or no data available."

        df['MA9'] = df['Close'].rolling(window=9).mean()

        delta = df['Close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=14).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
        RS = gain / loss
        df['RSI14'] = 100 - (100 / (1 + RS))
        df.dropna(inplace=True)

        latest = df.iloc[-1]
        signal = ""

        if latest['RSI14'] < 30 and latest['Close'] > latest['MA9']:
            signal = " Buy Signal: RSI is low, price above MA9."
        elif latest['RSI14'] > 70 and latest['Close'] < latest['MA9']:
            signal = "Sell Signal: RSI is high, price below MA9."
        else:
            signal = "Hold / No clear signal."

        indicators = latest[['Close', 'MA9', 'RSI14']].round(2).to_dict()
        return indicators, signal

    except Exception as e:
        return None, f"Error fetching data: {str(e)}"

def execute_trade(stock_name, qty, action):
    global capital

    ticker = stocks[stock_name]
    df_today = yf.download(ticker, period="1d", progress=False)
    if df_today.empty:
        return "No price data available today."
    price = float(df_today['Close'].iloc[-1])
    total_cost = qty * price + COMMISSION

    if action == "Buy":
        if total_cost > capital:
            return "Not enough capital."
        if qty * price < MIN_TRADE_AMOUNT:
            return "Trade amount too low."
        if qty * price > STARTING_CAPITAL * MAX_POSITION_SIZE:
            return "Trade exceeds position size limit."

        if stock_name in portfolio:
            old_qty = portfolio[stock_name]['quantity']
            old_price = portfolio[stock_name]['avg_price']
            new_qty = old_qty + qty
            new_avg = (old_qty * old_price + qty * price) / new_qty
            portfolio[stock_name]['quantity'] = new_qty
            portfolio[stock_name]['avg_price'] = new_avg
        else:
            portfolio[stock_name] = {
                'quantity': qty,
                'avg_price': price,
                'buy_date': datetime.datetime.now()
            }

        capital -= total_cost
        transactions.append([datetime.datetime.now(), stock_name, action, qty, price, COMMISSION, 0, ""])
        return f"Bought {qty} shares of {stock_name} at ₹{price:.2f}."

    elif action == "Sell":
        if stock_name not in portfolio or portfolio[stock_name]['quantity'] < qty:
            return "Not enough shares to sell."

        buy_price = portfolio[stock_name]['avg_price']
        pnl = (price - buy_price) * qty - COMMISSION
        portfolio[stock_name]['quantity'] -= qty
        if portfolio[stock_name]['quantity'] == 0:
            del portfolio[stock_name]
        capital += qty * price - COMMISSION
        transactions.append([datetime.datetime.now(), stock_name, action, qty, price, COMMISSION, pnl, ""])
        return f"Sold {qty} shares of {stock_name} at ₹{price:.2f}, P&L: ₹{pnl:.2f}."

    return "Invalid action."

def portfolio_dashboard():
    rows = []
    total_value = capital
    best_stock, worst_stock = None, None
    max_return, min_return = -np.inf, np.inf
    win_count = 0

    for stock, data in portfolio.items():
        ticker = stocks[stock]
        df_today = yf.download(ticker, period="1d", progress=False)
        if df_today.empty:
            continue
        current_price = float(df_today['Close'].iloc[-1])
        qty = data['quantity']
        avg_price = data['avg_price']
        unrealized = (current_price - avg_price) * qty
        ret = (unrealized / (avg_price * qty)) * 100
        days_held = (datetime.datetime.now() - data['buy_date']).days

        total_value += qty * current_price
        if ret > max_return:
            max_return, best_stock = ret, stock
        if ret < min_return:
            min_return, worst_stock = ret, stock
        if unrealized > 0:
            win_count += 1

        rows.append([stock, qty, round(avg_price, 2), round(current_price, 2), round(unrealized, 2), round(ret, 2), days_held])

    win_rate = (win_count / len(portfolio)) * 100 if portfolio else 0
    df = pd.DataFrame(rows, columns=["Ticker", "Quantity", "Avg Buy Price", "Current Price", "Unrealized P&L", "% Return", "Days Held"])
    return df, f"₹{total_value:,.2f}", f"{win_rate:.2f}%", best_stock or "N/A", worst_stock or "N/A"

def show_transactions():
    df = pd.DataFrame(transactions, columns=["Date", "Ticker", "Action", "Quantity", "Price", "Commission", "P&L", "Notes"])
    return df

def analyze(ticker):
    techs, signal = get_signals(ticker)
    if techs is None:
        return pd.DataFrame([{"Error": signal}]), signal
    df = pd.DataFrame([techs])
    return df, signal

def trade(stock_name, qty, action):
    if qty <= 0:
        return "Quantity must be positive."
    return execute_trade(stock_name, int(qty), action)

def build_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## NIFTY 50 Stock Screener & Virtual Portfolio")

        with gr.Tab("Stock Analyzer"):
            stock_selector = gr.Dropdown(choices=list(stocks.keys()), value=list(stocks.keys())[0], label="Select Stock")
            analyze_btn = gr.Button("Analyze")
            indicator_output = gr.Dataframe(label="Technical Indicators")
            signal_output = gr.Textbox(label="Trade Signal")
            analyze_btn.click(
                fn=lambda stock_name: analyze(stocks[stock_name]),
                inputs=stock_selector,
                outputs=[indicator_output, signal_output]
            )

        with gr.Tab("Trade"):
            trade_stock = gr.Dropdown(choices=list(stocks.keys()), value=list(stocks.keys())[0], label="Select Stock")
            qty_input = gr.Number(label="Quantity", value=5, precision=0)
            trade_result = gr.Textbox(label="Trade Status")
            with gr.Row():
                buy_btn = gr.Button("Buy")
                sell_btn = gr.Button("Sell")
            buy_btn.click(fn=lambda s, q: trade(s, q, "Buy"), inputs=[trade_stock, qty_input], outputs=trade_result)
            sell_btn.click(fn=lambda s, q: trade(s, q, "Sell"), inputs=[trade_stock, qty_input], outputs=trade_result)

        with gr.Tab("Portfolio"):
            dash_btn = gr.Button("Refresh Portfolio")
            portfolio_output = gr.Dataframe(label="Holdings")
            portfolio_val = gr.Textbox(label="Total Portfolio Value")
            win_rate_box = gr.Textbox(label="Win Rate %")
            best_stock_box = gr.Textbox(label="Best Performer")
            worst_stock_box = gr.Textbox(label="Worst Performer")
            dash_btn.click(fn=portfolio_dashboard, inputs=[], outputs=[portfolio_output, portfolio_val, win_rate_box, best_stock_box, worst_stock_box])

        with gr.Tab("Transactions"):
            txn_btn = gr.Button("Show Transactions")
            txn_table = gr.Dataframe(label="Transaction History")
            txn_btn.click(fn=show_transactions, inputs=[], outputs=txn_table)

    return demo

demo = build_interface()
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7898
* Running on public URL: https://ee9d45bb4edbe5ca98.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
